In [16]:
from PIL import Image as pil
import numpy as np
import pandas as pd
from IPython.core.debugger import set_trace

%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F
from keras.preprocessing.image import ImageDataGenerator
from torchvision import models
from torchsummary import summary

import pickle

In [18]:
from torch_arrange import random_weight, zero_weight, flatten, check_accuracy, train

In [3]:
LOADED = False

In [4]:
if LOADED == False:
    fname_Xaug = 'Xaug.pkl'
    fname_Yaug = 'Yaug.pkl'
    fname_Xtest = 'Xtest.pkl'
    fname_Ytest = 'Ytest.pkl'
    X_train_new = pickle.load(open(fname_Xaug,'rb'),)
    Y_train_new = pickle.load(open(fname_Yaug,'rb'),)
    X_test = pickle.load(open(fname_Xtest,'rb'),)
    y_test = pickle.load(open(fname_Ytest,'rb'),)
    LOADED = True
else :
    pass

In [5]:
model = models.resnet18(pretrained=False,)

In [6]:
model.avgpool = nn.AvgPool2d(kernel_size=3, stride=1, padding=0)

In [7]:
model.fc = nn.Linear(in_features=512, out_features=3, bias=True)

In [8]:
summary(model, (3, 80, 80))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 40, 40]           9,408
       BatchNorm2d-2           [-1, 64, 40, 40]             128
              ReLU-3           [-1, 64, 40, 40]               0
         MaxPool2d-4           [-1, 64, 20, 20]               0
            Conv2d-5           [-1, 64, 20, 20]          36,864
       BatchNorm2d-6           [-1, 64, 20, 20]             128
              ReLU-7           [-1, 64, 20, 20]               0
            Conv2d-8           [-1, 64, 20, 20]          36,864
       BatchNorm2d-9           [-1, 64, 20, 20]             128
             ReLU-10           [-1, 64, 20, 20]               0
       BasicBlock-11           [-1, 64, 20, 20]               0
           Conv2d-12           [-1, 64, 20, 20]          36,864
      BatchNorm2d-13           [-1, 64, 20, 20]             128
             ReLU-14           [-1, 64,

In [9]:
# 모든 parameter TRAINABLE 상태로 변경
for param in model.parameters():
    param.requires_grad = True

In [10]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [11]:
a = torch.from_numpy(X_train_new) 
b = torch.from_numpy(Y_train_new)
train_data = TensorDataset(a, b)

NUM_TRAIN = 20000
train_loader = DataLoader(train_data,batch_size= 64, sampler = sampler.SubsetRandomSampler(range(NUM_TRAIN)))
val_loader = DataLoader(train_data,batch_size= 64, sampler = sampler.SubsetRandomSampler(range(NUM_TRAIN,21930)))

c = torch.from_numpy(X_test) 
d = torch.from_numpy(y_test)
test_data = TensorDataset(c, d)
test_loader = DataLoader(test_data,batch_size= 64)


In [12]:
USE_GPU = True
if USE_GPU == True and torch.cuda.is_available() == True :
    device = torch.device('cuda')
else : 
    device = torch.device('cpu')
    
dtype = torch.float32
print_every = 100
print('device is ', device)

device is  cpu


In [69]:
def random_weight(shape):
    """
    Create random Tensors for weights; setting requires_grad=True means that we
    want to compute gradients for these Tensors during the backward pass.
    We use Kaiming normalization: sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # conv weight [out_channel, in_channel, kH, kW]
    # randn is standard normal distribution generator. 
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True
    return w

def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

# create a weight of shape [3 x 5]
# you should see the type `torch.cuda.FloatTensor` if you use GPU. 
# Otherwise it should be `torch.FloatTensor`
random_weight((3, 5))

tensor([[-0.8325,  1.9568,  0.8192,  0.1374,  1.1156],
        [-0.3931,  0.1464,  0.7893,  0.1792, -0.6073],
        [-0.3999, -0.5234, -0.3062, -1.0564,  0.0190]], requires_grad=True)

In [70]:
def check_accuracy(loader, model, data = 'val'):
    if data == 'val':
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [71]:
def train(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(train_loader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
#             print(scores)
#             print(y)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(val_loader, model)
                print()

In [15]:
learning_rate = 1e-3

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Print training status every epoch: set print_every to a large number
print_every = 100


train(model,optimizer,epochs=1)

TypeError: train() got multiple values for argument 'epochs'

In [73]:
check_accuracy(test_loader, model, data = 'test')

Checking accuracy on test set
Got 67 / 82 correct (81.71)
